# Data preprocessing

## Load data

In [1]:
import gzip
interactions = {}
data = []
# Load data
org_id = '4932' # Change to 9606 for Human

with gzip.open(f'data/{org_id}.protein.links.v11.0.txt.gz', 'rt') as f:
    next(f) # Skip header
    for line in f:
        p1, p2, score = line.strip().split()
        if float(score) < 700: # Filter high confidence interactions
            continue
        if p1 not in interactions:
            interactions[p1] = set()
        if p2 not in interactions:
            interactions[p2] = set()
        if p2 not in interactions[p1]:
            interactions[p1].add(p2)
            interactions[p2].add(p1)
            data.append((p1, p2))

print('Total number of interactions:', len(data))
print('Total number of proteins:', len(interactions.keys()))


Total number of interactions: 119051
Total number of proteins: 6157


## Split training, validation and testing data


In [2]:
import numpy as np
import math

np.random.seed(seed=0) # Fix random seed for reproducibility
np.random.shuffle(data)
train_n = int(math.ceil(len(data) * 0.8))
valid_n = int(math.ceil(train_n * 0.8))
train_data = data[:valid_n]
valid_data = data[valid_n:train_n]
test_data = data[train_n:]
print('Number of training interactions:', len(train_data))
print('Number of validation interactions:', len(valid_data))
print('Number of testing interactions:', len(test_data))

Number of training interactions: 76193
Number of validation interactions: 19048
Number of testing interactions: 23810


## Save the data

In [3]:
def save(filename, data):
    with open(filename, 'w') as f:
        for p1, p2 in data:
            f.write(f'{p1}\t{p2}\n')
            f.write(f'{p2}\t{p1}\n')

save(f'data/train/{org_id}.protein.links.v11.0.txt', train_data)
save(f'data/valid/{org_id}.protein.links.v11.0.txt', valid_data)
save(f'data/test/{org_id}.protein.links.v11.0.txt', test_data)

## Generate negative interactions

In [4]:
import random
proteins =set ()
negatives = []
for (p1,p2) in data:
        proteins.add(p1)
        proteins.add(p2)
while len(negatives)<len(data):
        s = random.sample(proteins, 2)
        prot1= s[0]
        prot2= s[1]
        if (prot1,prot2) in negatives or (prot2,prot1) in negatives :
                 continue
        if prot1 not in interactions[prot2]:
                 negatives.append((prot1, prot2))
print('Total number of negative interactions:', len(negatives))
# Split negative data
neg_train_data = negatives[:valid_n]
neg_valid_data = negatives[valid_n:train_n]
neg_test_data = negatives[train_n:]
print('Number of negative training interactions:', len(neg_train_data))
print('Number of negative validation interactions:', len(neg_valid_data))
print('Number of negative testing interactions:', len(neg_test_data))
# Save negative data 
save(f'data/train/{org_id}.negative_interactions.txt', neg_train_data)
save(f'data/valid/{org_id}.negative_interactions.txt', neg_valid_data)
save(f'data/test/{org_id}.negative_interactions.txt', neg_test_data)

Total number of negative interactions: 119051
Number of negative training interactions: 76193
Number of negative validation interactions: 19048
Number of negative testing interactions: 23810


## Preprocess GO annotations
### Load id mapping between annotation database and StringDB IDs

In [5]:
mapping = {}
source = {'4932': 'SGD_ID', '9606': 'Ensembl_UniProt_AC'} # mapping source

with gzip.open(f'data/{org_id}.protein.aliases.v11.0.txt.gz', 'rt') as f:
    next(f) # Skip header
    for line in f:
        string_id, p_id, sources = line.strip().split('\t')
        if source[org_id] not in sources.split():
            continue
        if p_id not in mapping:
            mapping[p_id] = set()
        mapping[p_id].add(string_id)
print('Loaded mappings', len(mapping))

Loaded mappings 10616


### Load annotations

In [6]:
gaf_files = {'4932': 'sgd.gaf.gz', '9606': 'goa_human.gaf.gz'}
annotations = set()
with gzip.open(f'data/{gaf_files[org_id]}', 'rt') as f:
    for line in f:
        if line.startswith('!'): # Skip header
            continue
        it = line.strip().split('\t')
        p_id = it[1]
        go_id = it[4]
        if it[6] == 'IEA' or it[6] == 'ND': # Ignore predicted or no data annotations
            continue
        if p_id not in mapping: # Not in StringDB
            continue
        s_ids = mapping[p_id]
        for s_id in s_ids:
            annotations.add((s_id, go_id))
print('Number of annotations:', len(annotations))

# Save annotations
with open(f'data/train/{org_id}.annotation.txt', 'w') as f:
    for p_id, go_id in annotations:
        f.write(f'{p_id}\t{go_id}\n')

Number of annotations: 40577


## Generate Plain Training Data

In [7]:
import os

tdf = open(f'data/train/{org_id}.plain.nt', 'w')
# Load GO
with open('data/go.obo') as f:
    tid = ''
    for line in f:
        line = line.strip()
        if line.startswith('id:'):
            tid = line[4:]
        if not tid.startswith('GO:'):
            continue
        if line.startswith('is_a:'):
            tid2 = line[6:].split(' ! ')[0]
            tdf.write(f'<http://{tid}> <http://is_a> <http://{tid2}> .\n')
        if line.startswith('relationship:'):
            it = line[14:].split(' ! ')[0].split()
            tdf.write(f'<http://{tid}> <http://{it[0]}> <http://{it[1]}> .\n')

# Load interactions
with open(f'data/train/{org_id}.protein.links.v11.0.txt') as f:
    for line in f:
        it = line.strip().split()
        tdf.write(f'<http://{it[0]}> <http://interacts> <http://{it[1]}> .\n')

# Load annotations
with open(f'data/train/{org_id}.annotation.txt') as f:
    for line in f:
        it = line.strip().split()
        tdf.write(f'<http://{it[0]}> <http://hasFunction> <http://{it[1]}> .\n')

tdf.close()
if not os.path.exists('data/transe'):
    os.makedirs('data/transe')
! wc -l 'data/train/{org_id}.plain.nt'

287137 data/train/4932.plain.nt


## Generate Classes Training Data for ELEmbeddings

In [8]:
if not os.path.exists('data/elembeddings'):
    os.makedirs('data/elembeddings')
! groovy el-embeddings/GenerateTrainingDataClasses -i 'data/train/{org_id}.protein.links.v11.0.txt' -a 'data/train/{org_id}.annotation.txt' -o 'data/train/{org_id}.classes.owl'


error: Missing required options: g, o
usage: groovy
 -g,--input <arg>    Gene Ontology file
 -h,--help           this information
 -i,--input <arg>    input STRING file
 -o,--output <arg>   output file containing generated ontology


### Normalize training data classes into four normal forms

In [9]:

! groovy -cp el-embeddings/jar/jcel.jar el-embeddings/Normalizer.groovy -i 'data/train/{org_id}.classes.owl' -o 'data/train/{org_id}.classes-normalized.owl'



SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.
Ignoring SubObjectPropertyOf*(8 7):The translation map is incomplete. Item id was not found: '8'.
Ignoring SubObjectPropertyOf*(10 12):The translation map is incomplete. Item id was not found: '10'.
Ignoring SubObjectPropertyOf*(11 12):The translation map is incomplete. Item id was not found: '11'.


## Generate RDF Representation of ELEmbeddings training data

In [10]:

! rapper 'data/train/{org_id}.classes.owl' -o ntriples > 'data/train/{org_id}.classes-rdf.nt'


rapper: Parsing URI file:///home/kulmanm/KAUST/CBRC/machine-learning-with-ontologies/data/train/9606.classes.owl with parser rdfxml
rapper: Serializing with serializer ntriples
rapper: Parsing returned 4368800 triples


## Generate Onto/OPA2Vec-compatible associations

In [11]:
import re
# generate OPA2VEC compatible associations
with open(f'data/train/{org_id}.OPA_associations.txt', 'w') as f:
    for p_id, go_id in annotations:
        go_num = re.split ('[A-Z]|:+',go_id)
        f.write (str(p_id)+" \t"+"<http://purl.obolibrary.org/obo/GO_"+str(go_num[3])+">\n")

### Generate Ontology projection

In [1]:
!groovy opa2vec/ProcessOntology.groovy data/go.owl elk data/axioms_orig data/axioms_infer data/classes
!cat data/axioms_orig data/axioms_infer > data/axioms

SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.
Loading of Axioms ...
Loading ...
    1%
    2%
    3%
    4%
    5%
    6%
    8%
    9%
    10%
    11%
    12%
    14%
    15%
    16%
    17%
    18%
    19%
    20%
    21%
    23%
    24%
    25%
    27%
    28%
    29%
    30%
    32%
    33%
    34%
    35%
    37%
    38%
    39%
    40%
    41%
    43%
    44%
    45%
    46%
    47%
    48%
    49%
    50%
    51%
    52%
    53%
    54%
    56%
    57%
    58%
    59%
    60%
    61%
    62%
    64%
    65%
    67%
    68%
    70%
    71%
    72%
    73%
    74%
    75%
    76%
    78%
    79%
    80%
    82%
    83%
    85%
    86%
    87%
    89%
    91%
    93%
    95%
    97%
    99%
    ... finished
    ... finished
Property Saturation Initialization ...
    ... finished
Reflexive Property Computation ...
    ...

In [12]:
eq_dict={}
with open ("data/axioms") as f:
    for line in f:
        tokens = line.split(" ", 2)
        go_class = tokens[0]
        if go_class not in eq_dict:
            eq_dict[go_class]=[]
        relation = tokens[1]
        value = tokens[2]
        if (relation == 'EquivalentTo'):
            literals = value.split()
            for lit in literals:
                if ("GO_") in lit:
                    eq_dict[go_class].append(lit)

In [13]:
owlf = open(f'data/train/{org_id}.owl_graph.nt', 'w')

with open ("data/axioms") as f:
    for line in f:
        tokens = line.split(" ", 2)
        go_class = tokens[0]
        relation = tokens[1]
        value = tokens[2]
        literals = value.split()
        for lit in literals:
            if ("GO_") in lit:
                owlf.write(f'{go_class} {relation} {lit} .\n')
                if (relation == 'SubClassOf'):
                    owlf.write(f'{lit} SubClassOf_inv {go_class} .\n')
                for item in eq_dict[lit]:
                    owlf.write(f'{go_class} {relation} {item} .\n') 
                    if (relation == 'SubClassOf'):
                        owlf.write(f'{item} SubClassOf_inv {go_class} .\n')


with open(f'data/train/{org_id}.protein.links.v11.0.txt') as f:
    for line in f:
        it = line.strip().split()
        owlf.write(f'<http://{it[0]}> <http://interacts> <http://{it[1]}> .\n')


with open(f'data/train/{org_id}.annotation.txt') as f:
    for line in f:
        it = line.strip().split()
        owlf.write(f'<http://{it[0]}> <http://hasFunction> <http://purl.obolibrary.org/obo/{it[1]}> .\n')

owlf.close()                   